# Imports

In [1]:
import math
import os
import numpy as np
import tensorflow as tf

from neuraxle.api.flask import FlaskRestApiWrapper
from neuraxle.base import ExecutionContext, DEFAULT_CACHE_FOLDER, ExecutionMode, BaseStep
from neuraxle.data_container import DataContainer
from neuraxle.hyperparams.space import HyperparameterSamples
from neuraxle.steps.numpy import OneHotEncoder
from neuraxle.pipeline import MiniBatchSequentialPipeline, Joiner
from neuraxle.steps.output_handlers import OutputTransformerWrapper
from neuraxle_tensorflow.tensorflow_v1 import TensorflowV1ModelStep

from steps.custom_json_decoder_for_2darray import CustomJSONDecoderFor2DArray
from steps.custom_json_encoder_of_outputs import CustomJSONEncoderOfOutputs

from data_reading import DATASET_PATH, TRAIN, TEST, X_train_signals_paths, load_X, load_y, \
    TRAIN_FILE_NAME, TEST_FILE_NAME

# Download Data

In [2]:
# Note: Linux bash commands start with a "!" inside those "ipython notebook" cells

DATA_PATH = "data/"

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

!python download_dataset.py

!pwd && ls
os.chdir("..")
!pwd && ls

DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)

/home/alexandre/Documents/LSTM-Human-Activity-Recognition
1_train_and_save_LSTM.ipynb	data_reading.py      pipeline.py       steps
2_call_rest_api_and_eval.ipynb	LICENSE		     __pycache__       venv
cache				model.py	     README.md
data				neuraxle_tensorflow  requirements.txt
/home/alexandre/Documents/LSTM-Human-Activity-Recognition/data
 download_dataset.py   source.txt	 'UCI HAR Dataset.zip'
 __MACOSX	      'UCI HAR Dataset'

Downloading...
Dataset already downloaded. Did not download twice.

Extracting...
Dataset already extracted. Did not extract twice.

/home/alexandre/Documents/LSTM-Human-Activity-Recognition/data
 download_dataset.py   source.txt	 'UCI HAR Dataset.zip'
 __MACOSX	      'UCI HAR Dataset'
/home/alexandre/Documents/LSTM-Human-Activity-Recognition
1_train_and_save_LSTM.ipynb	data_reading.py      pipeline.py       steps
2_call_rest_api_and_eval.ipynb	LICENSE		     __pycache__       venv
cache				model.py	     README.md
data				neuraxle_tensorflow  requirements.txt

D

# Load data

In [3]:
# Load "X" (the neural network's training and testing inputs)

X_train = load_X(X_train_signals_paths)
# X_test = load_X(X_test_signals_paths)

# Load "y" (the neural network's training and testing outputs)

y_train_path = os.path.join(DATASET_PATH, TRAIN, TRAIN_FILE_NAME)
# y_test_path = os.path.join(DATASET_PATH, TEST, TEST_FILE_NAME)

y_train = load_y(y_train_path)
# y_test = load_y(y_test_path)

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_train.shape, y_train.shape, np.mean(X_train), np.std(X_train))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(7352, 128, 9) (7352, 1) 0.10206611 0.40216514
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


# Create LSTM RNN Tensorflow Graph

In [4]:
def create_graph(step: TensorflowV1ModelStep):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters.
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network.
    # Note, some code of this notebook is inspired from an slightly different
    # RNN architecture used on another dataset, some of the credits goes to
    # "aymericdamien" under the MIT license.
    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)

    # Graph input/output
    data_inputs = tf.placeholder(tf.float32, [None, step.hyperparams['n_steps'], step.hyperparams['n_inputs']],
                                 name='data_inputs')
    expected_outputs = tf.placeholder(tf.float32, [None, step.hyperparams['n_classes']], name='expected_outputs')

    # Graph weights
    weights = {
        'hidden': tf.Variable(
            tf.random_normal([step.hyperparams['n_inputs'], step.hyperparams['n_hidden']])
        ),  # Hidden layer weights
        'out': tf.Variable(
            tf.random_normal([step.hyperparams['n_hidden'], step.hyperparams['n_classes']], mean=1.0)
        )
    }

    biases = {
        'hidden': tf.Variable(
            tf.random_normal([step.hyperparams['n_hidden']])
        ),
        'out': tf.Variable(
            tf.random_normal([step.hyperparams['n_classes']])
        )
    }

    data_inputs = tf.transpose(
        data_inputs,
        [1, 0, 2])  # permute n_steps and batch_size

    # Reshape to prepare input to hidden activation
    data_inputs = tf.reshape(data_inputs, [-1, step.hyperparams['n_inputs']])
    # new shape: (n_steps*batch_size, n_input)

    # ReLU activation, thanks to Yu Zhao for adding this improvement here:
    _X = tf.nn.relu(
        tf.matmul(data_inputs, weights['hidden']) + biases['hidden']
    )

    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, step.hyperparams['n_steps'], 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(step.hyperparams['n_hidden'], forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(step.hyperparams['n_hidden'], forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)

    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    return tf.matmul(lstm_last_output, weights['out']) + biases['out']

# Create Optimizer

In [5]:
def create_optimizer(step: TensorflowV1ModelStep):
    return tf.train.AdamOptimizer(learning_rate=step.hyperparams['learning_rate'])

# Create Loss

In [6]:
def create_loss(step: TensorflowV1ModelStep):
    # Loss, optimizer and evaluation
    # L2 loss prevents this overkill neural network to overfit the data
    l2 = step.hyperparams['lambda_loss_amount'] * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())

    # Softmax loss
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=step['expected_outputs'],
            logits=step['output']
        )
    ) + l2

# Create TensorflowV1ModelStep 

In [7]:
class ClassificationRNNTensorFlowModel(TensorflowV1ModelStep):
    def setup(self) -> BaseStep:
        TensorflowV1ModelStep.setup(self)

        self.losses = []
        self.accuracies = []

        return self

    def _will_process(self, data_container: DataContainer, context: ExecutionContext) -> ('BaseStep', DataContainer):
        if not isinstance(data_container.data_inputs, np.ndarray):
            data_container.data_inputs = np.array(data_container.data_inputs)

        if data_container.expected_outputs is not None:
            if not isinstance(data_container.expected_outputs, np.ndarray):
                data_container.expected_outputs = np.array(data_container.expected_outputs)

            if data_container.expected_outputs.shape != (len(data_container.data_inputs), self.hyperparams['n_classes']):
                data_container.expected_outputs = np.reshape(data_container.expected_outputs, (len(data_container.data_inputs), self.hyperparams['n_classes']))

        return data_container, context

    def _did_fit_transform(self, data_container: DataContainer, context: ExecutionContext) -> DataContainer:
        accuracy = np.mean(np.argmax(data_container.data_inputs, axis=1) == np.argmax(data_container.expected_outputs, axis=1))

        self.accuracies.append(accuracy)
        self.losses.append(self.loss)

        print("Batch Loss = " + "{:.6f}".format(self.losses[-1]) + ", Accuracy = {}".format(self.accuracies[-1]))

        return data_container

# Neuraxle Pipeline 

In [8]:
N_HIDDEN = 32
N_STEPS = 128
N_INPUTS = 9
LAMBDA_LOSS_AMOUNT = 0.0015
LEARNING_RATE = 0.0025
N_CLASSES = 6
BATCH_SIZE = 1500


class HumanActivityRecognitionPipeline(MiniBatchSequentialPipeline):
    def __init__(self):
        MiniBatchSequentialPipeline.__init__(self, [
            OutputTransformerWrapper(OneHotEncoder(nb_columns=N_CLASSES, name='one_hot_encoded_label')),
            ClassificationRNNTensorFlowModel(
                create_graph=create_graph,
                create_loss=create_loss,
                create_optimizer=create_optimizer
            ).set_hyperparams(
                HyperparameterSamples({
                    'n_steps': N_STEPS,
                    'n_inputs': N_INPUTS, 
                    'n_hidden': N_HIDDEN,
                    'n_classes': N_CLASSES,
                    'learning_rate': LEARNING_RATE,
                    'lambda_loss_amount': LAMBDA_LOSS_AMOUNT,
                    'batch_size': BATCH_SIZE
                })
            ),
            Joiner(batch_size=BATCH_SIZE)
        ])

# Train Pipeline 

In [9]:
training_data_count = len(X_train)
training_iters = training_data_count * 3

pipeline = HumanActivityRecognitionPipeline()

no_iter = int(math.floor(training_iters / BATCH_SIZE))
for _ in range(no_iter):
    pipeline, outputs = pipeline.fit_transform(X_train, y_train)

pipeline.save(ExecutionContext(DEFAULT_CACHE_FOLDER))

pipeline.teardown()





Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for u

HumanActivityRecognitionPipeline
(
	HumanActivityRecognitionPipeline(
	name=HumanActivityRecognitionPipeline,
	hyperparameters=HyperparameterSamples()
)(
		[('OutputTransformerWrapper',
  OutputTransformerWrapper(
	wrapped=OneHotEncoder(
	name=one_hot_encoded_label,
	hyperparameters=HyperparameterSamples()
),
	hyperparameters=HyperparameterSamples()
)),
 ('ClassificationRNNTensorFlowModel',
  ClassificationRNNTensorFlowModel(
	name=ClassificationRNNTensorFlowModel,
	hyperparameters=HyperparameterSamples([('n_steps', 128),
                       ('n_inputs', 9),
                       ('n_hidden', 32),
                       ('n_classes', 6),
                       ('learning_rate', 0.0025),
                       ('lambda_loss_amount', 0.0015),
                       ('batch_size', 1500)])
)),
 ('Joiner', Joiner(
	name=Joiner,
	hyperparameters=HyperparameterSamples()
))]	
)
)

# Serve Rest Api

In [10]:
pipeline = HumanActivityRecognitionPipeline()

pipeline = pipeline.load(ExecutionContext(DEFAULT_CACHE_FOLDER))

# pipeline, outputs = pipeline.fit_transform(X_train, y_train)  # we could train further more here for instance.

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

INFO:tensorflow:Restoring parameters from /home/alexandre/Documents/LSTM-Human-Activity-Recognition/cache/HumanActivityRecognitionPipeline/ClassificationRNNTensorFlowModel/ClassificationRNNTensorFlowModel.ckpt


In [ ]:
app = FlaskRestApiWrapper(
    json_decoder=CustomJSONDecoderFor2DArray(),
    wrapped=pipeline,
    json_encoder=CustomJSONEncoderOfOutputs()
).get_app()

app.run(debug=False, port=5000)

 * Serving Flask app "neuraxle.api.flask" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
